In [2]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
import time
from pandas.io.json import json_normalize
from copy import deepcopy

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [3]:
### MULTI-CANDIDATE SEARCH ###
# DEVELOPING LOOP FOR CANDIDATE IDS #

#Set search for all 2020 (two-year transaction period) Maine candidates
cand_state = 'ME'
period='2020'
parameters = {'election_year':period
            ,'state':cand_state
            ,'api_key':config.api_key}
#Requests candidate info
r_cands = requests.get('https://api.open.fec.gov/v1/candidates/search',params=parameters).json()
r_cands

#Locates and sets Committee ID from 'principal_committees' sub-array
comm_ids=json_normalize(data=r_cands['results'],record_path='principal_committees')[['committee_id','name','party']]
comm_ids

,committee_id,name,party
0,C00710087,MICHAEL BUNKER FOR MAINE,DEM
1,C00314575,COLLINS FOR SENATOR,REP
2,C00709899,SARA GIDEON FOR MAINE,DEM
3,C00653816,COMMITTEE TO ELECT JARED GOLDEN,DEM
4,C00703413,FRIENDS OF BRE KIDMAN,DEM
5,C00706739,DEREK FOR U.S. SENATE,REP
6,C00683193,DR CAT LONDON FOR US SENATE,DEM
7,C00433391,PINGREE FOR CONGRESS,DEM
8,C00709279,SWEET FOR SENATE,DEM
9,C00694364,DANIELLE VANHELSING FOR SENATE,IND


In [456]:
#Initialize dataframe collector
dfs=[]

#Initialize last_index and last_date
querydict = {'per_page':'100'
              ,'committee_id':comm_id['committee_id'][0]
              ,'sort':'contribution_receipt_date'
              ,'api_key':config.api_key
              ,'is_individual':'true'
              ,'two_year_transaction_period':period
       }

r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()

#Results rows
df = json_normalize(r['results'])
dfs.append(df)

#Last page variables
last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']

    for comm_id['committee_id'] in comm_id:

    #Loop through pages
    while r['pagination']['last_indexes'] is not None:

        #Initialize query, without last_index and date
        querydict = {'per_page':'100'
                  ,'committee_id':comm_id['committee_id']
                  ,'sort':'contribution_receipt_date'
                  ,'api_key':config.api_key
                  ,'is_individual':'true'
                  ,'two_year_transaction_period':period
                  ,'last_index':last_index
                  ,'last_contribution_receipt_date':last_date
           }

        r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()

        #Results rows
        df = json_normalize(r['results'])
        dfs.append(df)

        #Pagination variables
        if r['pagination']['last_indexes'] is None:
            None
        else: 
            last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
            last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']        

        dfs.append(df)


#Concatenate all dfs
contribs=pd.concat(dfs,sort=True)
contribs=contribs.drop_duplicates(subset='transaction_id')

In [458]:
#Write itemized individual results to a CSV
cwd = os.getcwd()
contribs.to_csv(cwd+'/collins-itemized-receipts.csv')